In [ ]:
# Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import importlib.util
import os

In [ ]:
script_path = '/content/drive/MyDrive/DS Project - Moderately/code/2_Model_Training_v2.py'

spec = importlib.util.spec_from_file_location("model_training", script_path)
model_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model_module)

main = model_module.main

from argparse import Namespace

In [ ]:
args = Namespace(
    data_file='/content/drive/MyDrive/DS Project - Moderately/data/babe_polibias_combined_labeled.csv',
    output_dir='/content/drive/MyDrive/DS Project - Moderately/moderately_artifacts',
    batch_size=16,
    max_length=128,
    epochs=5,
    lr=2.16411857404266e-05,
    weight_decay=0.01,
    warmup_ratio=0.1,
    alpha=0.5848983419474759,
    test_size=0.15,
    seed=42
)

In [ ]:
main(args)

Loaded DataFrame: (4779, 4)
                                                text  ideology  factuality  \
0  NYPD Commissioner Dermot Shea on Monday expres...         3           0   
1  School systems across the country are adopting...         1           2   
2  And then along came President Barry Obama, who...         1           2   
3  The curfews, which have never before occurred ...         3           0   
4  Rather than help be a part of the solution, Tr...         4           2   

  data_source  
0        babe  
1        babe  
2        babe  
3        babe  
4        babe   

After dropna: (4779, 4)
Ideology distribution:
 ideology
2    0.377485
3    0.197112
0    0.147520
1    0.146474
4    0.131408
Name: proportion, dtype: float64
Factuality distribution:
 factuality
0    0.373091
1    0.313873
2    0.313036
Name: proportion, dtype: float64 

Splits → Train: (3345, 4), Val: (717, 4), Test: (717, 4)

Computed class weights.

Starting training...



Epoch 1/5:   0%|          | 0/210 [00:00<?, ?it/s]


Epoch 1 avg training loss: 1.2031
 → Val Ideo Acc 0.4979, F1 0.4420 | Val Fact Acc 0.7183, F1 0.7093



Epoch 2/5:   0%|          | 0/210 [00:00<?, ?it/s]


Epoch 2 avg training loss: 0.8880
 → Val Ideo Acc 0.5579, F1 0.5040 | Val Fact Acc 0.7587, F1 0.7529



...


Epoch 3 avg training loss: 0.6709
 → Val Ideo Acc 0.5370, F1 0.4945 | Val Fact Acc 0.7545, F1 0.7531



Epoch 4/5:   0%|          | 0/210 [00:00<?, ?it/s]


Epoch 4 avg training loss: 0.5061
 → Val Ideo Acc 0.5704, F1 0.5255 | Val Fact Acc 0.7727, F1 0.7691



Epoch 5/5:   0%|          | 0/210 [00:00<?, ?it/s]


Epoch 5 avg training loss: 0.4081
 → Val Ideo Acc 0.5746, F1 0.5294 | Val Fact Acc 0.7671, F1 0.7645

Evaluating on test set...
Test Ideo Acc 0.5690, F1 0.5226 | Test Fact Acc 0.7503, F1 0.7458

Artifacts saved to /content/drive/MyDrive/DS Project - Moderately/moderately_artifacts


## Scratch

In [ ]:
# !pip install -q transformers scikit-learn tqdm

In [ ]:
# Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import importlib.util, os, pandas as pd, numpy as np, torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.optim import AdamW
from transformers import DistilBertTokenizerFast, get_linear_schedule_with_warmup
from tqdm.auto import tqdm

In [ ]:
# Load training script
script_path = '/content/drive/MyDrive/DS Project - Moderately/code/2_Model_Training_v2.py'
spec = importlib.util.spec_from_file_location("model_training", script_path)
model_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model_module)

In [ ]:
# Extract utilities from script
set_seed = model_module.set_seed
prepare_dataloader = model_module.prepare_dataloader
MultiTaskDistilBert = model_module.MultiTaskDistilBert

AttributeError: module 'model_training' has no attribute 'set_seed'

In [ ]:
# # Define arguments for training
# args = Namespace(
#     data_file='/content/drive/MyDrive/DS Project - Moderately/data/babe_polibias_combined_labeled.csv',
#     output_dir='/content/drive/MyDrive/DS Project - Moderately/moderately_artifacts',
#     batch_size=16,
#     max_length=128,
#     epochs=7,
#     lr=2.16411857404266e-05,
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     alpha=0.5848983419474759,
#     test_size=0.15,
#     seed=42
# )

# # Run training
# main(args)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Epoch 1/7 - loss: 1.2179


KeyboardInterrupt: 

In [ ]:
# Define arguments
class Args: pass
args = Args()
args.data_file   = '/content/drive/MyDrive/DS Project - Moderately/data/babe_polibias_combined_labeled.csv'
args.test_size   = 0.15
args.seed        = 42
args.batch_size  = 16
args.max_length  = 128
args.epochs      = 7
args.lr          = 2.16411857404266e-05
args.weight_decay= 0.01
args.warmup_ratio= 0.1
args.alpha       = 0.5848983419474759

# Load and shuffle
df = pd.read_csv(args.data_file).dropna(subset=['text','ideology','factuality'])
df = df.sample(frac=1, random_state=args.seed).reset_index(drop=True)
print("Data shape:", df.shape)
print(df.head())

Data shape: (4779, 4)
                                                text  ideology  factuality  \
0  Nevertheless, The Jesus Is King crooner launch...         1           1   
1  What appears to have happened is that, like Ba...         4           2   
2  Radical Virginia Citizens Defense League has o...         3           1   
3  Five of eight gun control bills endorsed by De...         2           0   
4  Here are three obvious but important ways the ...         0           2   

  data_source  
0        babe  
1        babe  
2        babe  
3        babe  
4        babe  


In [ ]:
# Split and encode labels
train_val_df, test_df = train_test_split(
    df, test_size=args.test_size, stratify=df['ideology'], random_state=args.seed
)

ideo_enc = LabelEncoder().fit(train_val_df['ideology'])
fact_enc = LabelEncoder().fit(train_val_df['factuality'])
for split in (train_val_df, test_df):
    split['ideology_label']   = ideo_enc.transform(split['ideology'])
    split['factuality_label'] = fact_enc.transform(split['factuality'])

In [ ]:
# Compute class weights
num_ideo = len(ideo_enc.classes_)
num_fact = len(fact_enc.classes_)

ideo_weights = compute_class_weight(
    'balanced', classes=np.arange(num_ideo), y=train_val_df['ideology_label'].to_numpy()
)
fact_weights = compute_class_weight(
    'balanced', classes=np.arange(num_fact), y=train_val_df['factuality_label'].to_numpy()
)

In [ ]:
# Move weights to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ideo_weights = torch.tensor(ideo_weights, dtype=torch.float).to(device)
fact_weights = torch.tensor(fact_weights, dtype=torch.float).to(device)

In [ ]:
# Prepare tokenization & DataLoaders
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_loader = prepare_dataloader(train_val_df, tokenizer, args.max_length, args.batch_size)
test_loader  = prepare_dataloader(test_df,     tokenizer, args.max_length, args.batch_size)
print(f"Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")

Train batches: 254, Test batches: 45


In [ ]:
# Build model, optimizer, and scheduler
set_seed(args.seed)
model = MultiTaskDistilBert(
    num_ideo_labels   = num_ideo,
    num_fact_labels   = num_fact,
    weight_ideo       = ideo_weights,
    weight_fact       = fact_weights,
    alpha             = args.alpha,
    beta              = 1.0 - args.alpha
).to(device)
optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
total_steps = len(train_loader) * args.epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = int(args.warmup_ratio * total_steps),
    num_training_steps = total_steps
)
print(model)

MultiTaskDistilBert(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [ ]:
# Training loop with progress bar
for epoch in range(1, args.epochs + 1):
    model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch {epoch}")
    for i, (input_ids, attention_mask, ideo_lbl, fact_lbl) in enumerate(loop, start=1):
        input_ids      = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        ideo_lbl       = ideo_lbl.to(device)
        fact_lbl       = fact_lbl.to(device)

        optimizer.zero_grad()
        out = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            ideology_label=ideo_lbl,
            factuality_label=fact_lbl
        )
        loss = out['loss']
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        loop.set_postfix(loss=(running_loss / i))
    print(f"Epoch {epoch} avg loss: {running_loss/len(train_loader):.4f}")

Epoch 1:   0%|          | 0/254 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Evaluation on test set
def eval_epoch(loader):
    model.eval()
    all_ideo_preds, all_ideo_labels = [], []
    all_fact_preds, all_fact_labels = [], []
    with torch.no_grad():
        for input_ids, attention_mask, ideo_lbl, fact_lbl in loader:
            input_ids      = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            out = model(input_ids=input_ids, attention_mask=attention_mask)
            all_ideo_preds  += out['ideology_logits'].argmax(dim=-1).cpu().tolist()
            all_fact_preds  += out['factuality_logits'].argmax(dim=-1).cpu().tolist()
            all_ideo_labels += ideo_lbl.tolist()
            all_fact_labels += fact_lbl.tolist()
    from sklearn.metrics import accuracy_score, f1_score
    ideo_acc = accuracy_score(all_ideo_labels, all_ideo_preds)
    fact_acc = accuracy_score(all_fact_labels, all_fact_preds)
    ideo_f1  = f1_score(all_ideo_labels, all_ideo_preds, average='macro')
    fact_f1  = f1_score(all_fact_labels, all_fact_preds, average='macro')
    return {'ideo_acc': ideo_acc, 'fact_acc': fact_acc, 'ideo_f1': ideo_f1, 'fact_f1': fact_f1}

metrics = eval_epoch(test_loader)
print("Test metrics:", metrics)

In [ ]:
# Save artifacts
artifact_dir = '/content/drive/MyDrive/DS Project - Moderately/moderately_artifacts'
os.makedirs(artifact_dir, exist_ok=True)
model.save_pretrained(os.path.join(artifact_dir,'model'))
tokenizer.save_pretrained(os.path.join(artifact_dir,'tokenizer'))
with open(os.path.join(artifact_dir,'ideo_encoder.pkl'),'wb') as f: pickle.dump(ideo_enc, f)
with open(os.path.join(artifact_dir,'fact_encoder.pkl'),'wb') as f: pickle.dump(fact_enc, f)
print("Artifacts saved to", artifact_dir)